# An Introduction to SunPy

This session aims to give a rapid and high level interview to the SunPy core package. We will touch on a lot of things in a little detail, we will follow up in more detail in subsequent sessions.

### Outline

* Common imports
* Astropy units
* Downloading an AIA image
* Loading data into sunpy's Map
* Accessing data and metadata from Map
* World coordinates with astropy and sunpy
* Visualising Maps in world coordinates

# Download an AIA Image

###  Importing Packages

In [ ]:
import astropy.units as u
from sunpy.net import Fido, attrs as a

### Using Astropy Units

In [ ]:
10 * u.m

In [ ]:
wave = 171 * u.AA
wave

In [ ]:
wave.to(u.nm)

In [ ]:
wave + 1 * u.m

### A Simple Search with Fido

### Search "attrs"

Sunpy search uses the concept of "attrs" which can be combined together with logical operations to form complex queries. We are going to do a very simple query today, we will go through more complex uses on Wednesday.

The most obvious attr is `a.Instrument` which lets you select which instrument you are interested in data from:

In [ ]:
a.Instrument

Some attrs like `a.Instrument` have a list of fixed values, and you can inspect what these are, and auto complete them by pressing tab. Others take values, such as time.

In [ ]:
results = Fido.search(a.Time("2011/06/07 07:00", "2011/06/07 07:01"),
                      a.Sample(1*u.min),
                      a.Instrument.aia,
                      a.Wavelength(wave))

In [ ]:
results

In [ ]:
files = Fido.fetch(results)

In [ ]:
files

## Loading images with Map

In [ ]:
import sunpy.map

In [ ]:
aiamap = sunpy.map.Map(files)

In [ ]:
aiamap

### Useful metadata

In [ ]:
aiamap.date

In [ ]:
aiamap.name

In [ ]:
aiamap.wavelength

In [ ]:
aiamap.scale

### Coordinate information

Maps are fully coordinate aware, their primary goal is to enable inspection and operations on both the array and the coordinate information at the same time.

sunpy makes heavy use of the [astropy coordinates framework](https://docs.astropy.org/en/stable/coordinates/index.html), which allows representation of and transformations between world coordinates.

Each map has a `.coordinate_frame` attribute, which is a description of the world coordinates of the image. This, as we will see, can be used to describe positions in world coordinates and compare them to the image.

In [ ]:
aiamap.coordinate_frame

This frame is in helioprojective coordinates, it has information on the radius of the Sun, the time of the observation and the position of the observer.

In [ ]:
aiamap.observer_coordinate

## Visualising a Map

sunpy uses matplotlib and astropy's [`wcsaxes`](https://docs.astropy.org/en/stable/visualization/wcsaxes/index.html) module to integrate the coordinates system with matplotlib.

In [ ]:
# If this doesn't work or you don't see plots try %matplotlib inline
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ax = plt.subplot(projection=aiamap)
im = aiamap.plot()
plt.colorbar()

### Overlaying a Heliographic grid

In [ ]:
plt.figure()
ax = plt.subplot(projection=aiamap)
im = aiamap.plot()
grid = aiamap.draw_grid()

### Overplotting Points in World Coordinates

We can use the `.coordinate_frame` property to get the information about the coordinates represented by the map. This includes, the coordinate type, observation time, and observer position.

In [ ]:
from astropy.coordinates import SkyCoord

In [ ]:
point = SkyCoord(Tx=700*u.arcsec, Ty=-400*u.arcsec, frame=aiamap.coordinate_frame)

In [ ]:
point

In [ ]:
plt.figure()
ax = plt.subplot(projection=aiamap)
im = aiamap.plot()
ax.plot_coord(point, "o")

#### Overplotting In Heliographic Coordinates

In [ ]:
point = SkyCoord(lon=60*u.deg, lat=-25*u.deg, frame='heliographic_stonyhurst')

In [ ]:
point

In [ ]:
plt.figure()
ax = plt.subplot(projection=aiamap)
im = aiamap.plot()
ax.plot_coord(point, "o")